In [1]:
# Data manipulation
import pandas as pd
import numpy as np
import re

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

In [2]:
# Import the csv

hedge = pd.read_csv('hedge_volume_new.csv')
production = pd.read_csv('production_filledna_clean.csv')

In [3]:
# Trim and rename the columns

hedge = hedge[['tic', 'file_header', 'maturity_year', 'maturity_year_old', 'product', 
               'hedge_volume', 'unit_hedgevol', 'volume_ann_multiplier', 'instrument']]

hedge.columns = ['ticker', 'file_header', 'maturity', 'detail_year',
                 'product', 'volume', 'unit', 'multiplier', 'instrument']

In [4]:
# Extract filing year

hedge['year'] = hedge['file_header'].apply(lambda x: int(re.findall('\d{4}', x)[0]))

In [5]:
# Observe unique values of maturity column, notice those bizzare ones, dtype = object instead of int

hedge['maturity'].unique()

array(['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2004', '2005', '2015', '2016', '2017', '2018', '2019',
       '2003', '201)', '202)', '20Q1', '20er', '2020', '20ar'],
      dtype=object)

In [6]:
# Fix bizzare maturity and coerce maturity column as integers

hedge.loc[hedge['maturity'].isin(['201)', '20Q1', '20ar', '202)', '20er']), 'maturity'] = \
[2012, 2012, 2012, 2013, 2014, 2010, 2010, 2010, 2010, 2015, 2008, 2009, 2014, 2014]

hedge['maturity'] = hedge['maturity'].astype(int)

In [7]:
# Only keep rows where 
# 1) Maturity year = hedging year
# 2) Instrument doesn't contain 'basis' or 'call'

hedge = hedge.loc[(hedge['maturity'] == hedge['year']) &
                  (~hedge['instrument'].str.contains('call|basis', na=False))].reset_index(drop=True)

In [8]:
# Get the unique detail_year information for daily data

hedge.loc[hedge['multiplier'] == 365]['detail_year'].unique()

array(['2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2013', '2014', '2015', '2016', '2017', 'q2|2017', 'q3 q4|2017',
       'first quarter 2003', 'second quarter 2003', 'third quarter 2003',
       'fourth quarter 2003', 'first quarter 2004', 'fourth quarter 2004',
       'second quarter 2004', 'third quarter 2004', 'first quarter 2005',
       'second quarter 2005', 'march 2005 - december 2005',
       'third quarter 2005', 'fourth quarter 2005',
       'april 2005 - december 2005', 'first quarter of 2006',
       'second quarter 2006', 'third quarter 2006', 'fourth quarter 2006',
       'april 2007 december 2007', 'fiscal 2007', 'calendar 2008',
       'january february 2008', 'march december 2008', 'calendar 2009',
       'calendar 2010', 'calendar 2011', 'calendar 2012(1)',
       'april 2012 - december 2012(1)', 'january 2012 - june 2012',
       'july 2012 - december 2012', 'calendar 2013', 'calendar 2014',
       'monthly calendar 2014', '1q 2009', '2q 

In [9]:
def converter(x):
    # Yearly production
    if x in ['2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
             '2013', '2014', '2015', '2016', '2017', 'fiscal 2007', 'calendar 2008'
             'calendar 2009', 'calendar 2010', 'calendar 2011', 'calendar 2012(1)',
             'calendar 2013', 'calendar 2014', 'january - december 2010', 'jan 08 - dec 08',
             'jan 09 - dec 09', 'jan 10 - dec 10', 'jan 11 - dec 11', 'jan 12 - dec 12',
             'jan 13 - dec 13', 'jan 14 - dec 14', 'jan 15 - dec 15', 'jan 16 - dec 16',
             'jan 17 - dec 17']:
        return 365
    
    # 2 months
    elif x in ['january february 2008', 'nov 17 - dec 17']:
        return 365/12*2
    
    # 10 months
    elif x in ['march december 2008', 'march through december of 2007', 'mar 12 - dec 12',
               'jan 17 - oct 17']:
        return 365/12*10
    
    # 9 months
    elif x in ['april 2012 - december 2012(1)', 'apr 17 - dec 17']:
        return 365/12*9
    
    # 6 months
    elif x in ['january 2012 - june 2012', 'july 2012 - december 2012', 'january - june 2010',
               'jan 14 - jun 14', 'jan 16 - jun 16']:
        return 365/12*6
    
    # 11 months
    elif x in ['feb 08 - dec 08', 'feb 11 - dec 11', 'feb 16 - dec 16']:
        return 365/12*11
    
    # 7 months
    elif x in ['apr 17 - oct 17']:
        return 365/12*7
    
    # Quarterly production
    else:
        return 365/12*4

In [10]:
# Apply the transformation

detail_year = hedge.loc[hedge['multiplier'] == 365]['detail_year']

hedge.loc[hedge['multiplier'] == 365, 'multiplier'] = detail_year.apply(converter)

In [11]:
# Convert all volume data to annual volume

hedge['volume'] = hedge['volume'] * hedge['multiplier']

production[['OIL_PRODUCTION', 'NGL_PRODUCTION', 'GAS_PRODUCTION']] = \
production[['OIL_PRODUCTION', 'NGL_PRODUCTION', 'GAS_PRODUCTION']] * 365

In [12]:
# Drop unnecessary columns

hedge.drop(['file_header', 'maturity', 'unit', 'instrument', 'multiplier', 'detail_year'], axis=1, inplace=True)

In [13]:
# Convert product names into more informative ones

product_convertion = {
    'gas': 'Natural Gas',
    'oil': 'Oil',
    'ngl': 'Natural Gas Liquid'
}

hedge['product'] = hedge['product'].map(product_convertion)

In [14]:
# Sum up the volumes for the same ticker, year and product

hedge = hedge.groupby(['ticker', 'year', 'product']).agg({'volume':'sum'}).reset_index()

In [15]:
# Merge the dataframes based on year and ticker

hedge_merge = pd.merge(hedge, production, on=['year', 'ticker'])

In [16]:
# Mutate a column according to production type, and calculate the hedge ratio

hedge_merge['production'] = np.where(hedge_merge['product'] == 'Natural Gas', hedge_merge['GAS_PRODUCTION'], 
         np.where(hedge_merge['product'] == 'Oil', hedge_merge['OIL_PRODUCTION'], hedge_merge['NGL_PRODUCTION']))

hedge_merge['hedge_ratio'] = hedge_merge['volume'] / hedge_merge['production']

In [17]:
# Trim out unnecessary columns

hedge_merge = hedge_merge[['ticker', 'year', 'product', 'volume', 'production', 'hedge_ratio']]

hedge_merge.sample(25)

,ticker,year,product,volume,production,hedge_ratio
145,UNT,2014,Oil,2646.250,3843.450,0.689
1,APA,2006,Oil,6064.000,81970.605,0.074
2,APA,2007,Natural Gas,79841.889,655667.020,0.122
109,RRC,2017,Oil,3117.830,4786.975,0.651
53,CLR,2014,Natural Gas,101105.211,114295.005,0.885
105,RRC,2016,Natural Gas Liquid,3923.385,27749.490,0.141
38,APC,2015,Natural Gas,283614.855,851910.000,0.333
94,RRC,2012,Oil,730.000,2843.350,0.257
13,APA,2012,Oil,16738.000,128489.490,0.130
92,RRC,2012,Natural Gas,147223.593,215962.835,0.682


In [18]:
# Affirm that no hedge ratio > 1

sum(hedge_merge['hedge_ratio'] > 1)
hedge_merge[hedge_merge['hedge_ratio'] > 1]

4

,ticker,year,product,volume,production,hedge_ratio
67,PDCE,2013,Natural Gas,28489.287,15431.470,1.846
69,PDCE,2014,Natural Gas,19455.115,19298.000,1.008
86,RRC,2008,Oil,3285.000,3076.220,1.068
88,RRC,2009,Oil,2920.000,2556.825,1.142


In [19]:
# Output the csv

hedge_merge.to_csv('hedge_ratio.csv', index=False)